In [7]:
import pandas as pd
import numpy as np
import os

import difflib
import unidecode

from datetime import datetime
import locale

## Union municipio y distritos con predicciones Modelo y geo
Debido a que los modelos de predicciones de series temporales no se cargarán en la aplicación, es nesario que estos esten cargados en BBDD. Es Script se encarga de leer el fichero con las predicciones y los valores actuales y unirlos con las zonas geográficas. Asi se tendrá un CSV don la información de los hisotricos en formato correcto y con el id de geo.

In [8]:
data_model = pd.read_csv('./predicciones_completas.csv')
data_model.head()

,Fecha,TipoValor,Valor,ConfLow,ConfUp,Distrito
0,2007-11-01,actual,4000.0,NaN,NaN,arganzuela
1,2007-12-01,actual,4000.0,NaN,NaN,arganzuela
2,2008-01-01,actual,4000.0,NaN,NaN,arganzuela
3,2008-02-01,actual,3824.0,NaN,NaN,arganzuela
4,2008-03-01,actual,4000.0,NaN,NaN,arganzuela


In [9]:
# Comprobamos que todos tienen fecha
data_model[data_model.Fecha.isna()]

,Fecha,TipoValor,Valor,ConfLow,ConfUp,Distrito


In [10]:
data_geo = pd.read_csv("./geo_elements.csv")
data_geo.head()

,id,name,st_astext,level_id
0,1,Madrid,"MULTIPOLYGON(((-3.88557654999994 40.57445562,-...",1
1,2,Villaverde,MULTIPOLYGON(((-3.70366015928759 40.3635648718...,2
2,3,Villa de Vallecas,MULTIPOLYGON(((-3.60705428494233 40.3866622239...,2
3,34,Gaztambide,MULTIPOLYGON(((-3.71111051636411 40.4389285909...,3
4,4,Usera,MULTIPOLYGON(((-3.68974605859619 40.3823311367...,2


In [11]:
# Vamos a ver simple vista si hay una gran diferencia en los nombres
distritos_idealista = data_model.Distrito.unique()
distritos_idealista

array(['arganzuela', 'barajas', 'carabanchel', 'centro', 'chamartin',
       'chamberi', 'ciudadlineal', 'fuencarral', 'hortaleza', 'latina',
       'madrid', 'moncloa', 'moratalaz', 'puentedevallecas', 'retiro',
       'salamanca', 'sanblas', 'tetuan', 'usera', 'vicalvaro',
       'villadevallecas', 'villaverde'], dtype=object)

In [12]:
data_geo = data_geo[data_geo.level_id < 3] 
data_geo.name.unique()

array(['Madrid', 'Villaverde', 'Villa de Vallecas', 'Usera',
       'Carabanchel', 'Moratalaz', 'Puente de Vallecas', 'Arganzuela',
       'Retiro', 'Vicálvaro', 'Latina', 'Moncloa - Aravaca', 'Centro',
       'Salamanca', 'Chamberí', 'Chamartín', 'San Blas - Canillejas',
       'Tetuán', 'Ciudad Lineal', 'Barajas', 'Hortaleza',
       'Fuencarral - El Pardo'], dtype=object)

In [13]:
# Mediante distancia de palabras vamos a mapear automaticamente todos los distritos
idealista_geo_names = {}
for name in distritos_idealista:
    idealista_geo_names[name] = difflib.get_close_matches(name, data_geo["name"].unique(), cutoff=0.3)[0]
    
idealista_geo_names

{'arganzuela': 'Arganzuela',
 'barajas': 'Barajas',
 'carabanchel': 'Carabanchel',
 'centro': 'Centro',
 'chamartin': 'Chamartín',
 'chamberi': 'Chamberí',
 'ciudadlineal': 'Ciudad Lineal',
 'fuencarral': 'Fuencarral - El Pardo',
 'hortaleza': 'Hortaleza',
 'latina': 'Latina',
 'madrid': 'Madrid',
 'moncloa': 'Salamanca',
 'moratalaz': 'Moratalaz',
 'puentedevallecas': 'Puente de Vallecas',
 'retiro': 'Retiro',
 'salamanca': 'Salamanca',
 'sanblas': 'Arganzuela',
 'tetuan': 'Tetuán',
 'usera': 'Usera',
 'vicalvaro': 'Vicálvaro',
 'villadevallecas': 'Villa de Vallecas',
 'villaverde': 'Villaverde'}

In [14]:
# Corregimos el caso de Moncloa y san-blas
idealista_geo_names["moncloa"] = "Moncloa - Aravaca"
idealista_geo_names["sanblas"] = "San Blas - Canillejas"

def get_geo_id(name):
    """
    Filter GeoZones by name and get id
    
    :param name: zone name to filter
    :return: zone id
    """
    return data_geo[data_geo.name == name].id.values[0]


data_model["geo_id"] = data_model["Distrito"].apply(lambda x: get_geo_id(idealista_geo_names[x]))
data_model.head()

,Fecha,TipoValor,Valor,ConfLow,ConfUp,Distrito,geo_id
0,2007-11-01,actual,4000.0,NaN,NaN,arganzuela,8
1,2007-12-01,actual,4000.0,NaN,NaN,arganzuela,8
2,2008-01-01,actual,4000.0,NaN,NaN,arganzuela,8
3,2008-02-01,actual,3824.0,NaN,NaN,arganzuela,8
4,2008-03-01,actual,4000.0,NaN,NaN,arganzuela,8


In [15]:
# Obtenemos los valores reales, sin las predicciones
data_idealista = pd.read_csv("./historico_madrid.csv")
# Nos quedamos con Madrid municipio y sus hijos
data_idealista = data_idealista[~data_idealista["Zona"].isin(["madrid-comunidad", "madrid-provincia"])]
data_idealista["geo_id"] = data_idealista["Zona"].apply(lambda x: get_geo_id(idealista_geo_names[x.replace("-", "")]))
data_idealista.head()

,Unnamed: 0,Mes,Precio m2,Variación mensual,Variación trimestral,Variación anual,Zona,geo_id
0,0,Abril 2021,2104.0,0.3,-1.7,-2.6,carabanchel,5
1,1,Marzo 2021,2098.0,-0.7,-3.1,-4.3,carabanchel,5
2,2,Febrero 2021,2112.0,-1.4,-2.6,-4.4,carabanchel,5
3,3,Enero 2021,2141.0,-1.1,-1.4,-2.1,carabanchel,5
4,4,Diciembre 2020,2165.0,-0.2,-0.3,-1.3,carabanchel,5


In [16]:
# Cambiamos el nombre de las columnas para machear contra las BD
# date, price, monthly_variation, quarterly_variation, annual_variation, geo_element_id
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8') 
data_idealista["date"] = data_idealista.Mes.apply(lambda x: datetime.strptime(x, '%B %Y').date())
# Mergeamos historico actual con geo
data_merged = data_idealista.merge(data_geo, left_on='geo_id', right_on='id')
# Formateamos tabla
data_merged = data_merged.drop(['Unnamed: 0', 'Mes', 'id'], axis=1)
data_merged = data_merged.reindex(columns=["Zona", "date", "Precio m2", "Variación mensual", "Variación trimestral", "Variación anual", "geo_id", "name", "st_astext", "level_id"])
data_merged.columns = ["idealista_name", "date", "price", "monthly_variation", "quarterly_variation", "annual_variation", "geo_element_id", "geo_name", "polygon", "geo_level"]
data_idealista = data_idealista.drop(['Mes'], axis=1)
data_idealista = data_idealista.reindex(columns=["date", "Precio m2", "Variación mensual", "Variación trimestral", "Variación anual", "Zona", "geo_id"])
data_idealista.columns = ["date", "price", "monthly_variation", "quarterly_variation", "annual_variation", "Distrito", "geo_element_id"]

In [17]:
# Comparamos estado de ambas tablas
data_idealista.sort_values(['Distrito', 'date']).head()

,date,price,monthly_variation,quarterly_variation,annual_variation,Distrito,geo_element_id
2391,2006-01-01,NaN,NaN,NaN,NaN,arganzuela,8
2390,2006-02-01,NaN,NaN,NaN,NaN,arganzuela,8
2389,2006-03-01,NaN,NaN,NaN,NaN,arganzuela,8
2388,2006-04-01,NaN,NaN,NaN,NaN,arganzuela,8
2387,2006-05-01,NaN,NaN,NaN,NaN,arganzuela,8


In [18]:
data_model.sort_values(['Distrito', 'Fecha']).head()

,Fecha,TipoValor,Valor,ConfLow,ConfUp,Distrito,geo_id
0,2007-11-01,actual,4000.0,NaN,NaN,arganzuela,8
1,2007-12-01,actual,4000.0,NaN,NaN,arganzuela,8
2,2008-01-01,actual,4000.0,NaN,NaN,arganzuela,8
3,2008-02-01,actual,3824.0,NaN,NaN,arganzuela,8
4,2008-03-01,actual,4000.0,NaN,NaN,arganzuela,8


In [19]:
# Mantenemos solo las predicciones de la tabla obtenida de los modelos
data_model = data_model[data_model["TipoValor"] != "actual"]
data_model.head()

,Fecha,TipoValor,Valor,ConfLow,ConfUp,Distrito,geo_id
162,2021-05-01,prediction,3964.0,3816.215907,4112.531351,arganzuela,8
163,2021-06-01,prediction,3972.0,3823.658580,4119.974024,arganzuela,8
164,2021-07-01,prediction,3973.0,3824.963067,4121.278511,arganzuela,8
165,2021-08-01,prediction,3957.0,3808.711239,4105.026683,arganzuela,8
166,2021-09-01,prediction,3893.0,3744.648015,4040.963459,arganzuela,8


In [20]:
# Añadimos en la tabla de valores reales las variables creadas por el modelo
data_idealista["conf_up"] = float('NaN')
data_idealista["conf_low"] = float('NaN')
data_idealista["value_type"] = "current"
data_idealista.head()

,date,price,monthly_variation,quarterly_variation,annual_variation,Distrito,geo_element_id,conf_up,conf_low,value_type
0,2021-04-01,2104.0,0.3,-1.7,-2.6,carabanchel,5,NaN,NaN,current
1,2021-03-01,2098.0,-0.7,-3.1,-4.3,carabanchel,5,NaN,NaN,current
2,2021-02-01,2112.0,-1.4,-2.6,-4.4,carabanchel,5,NaN,NaN,current
3,2021-01-01,2141.0,-1.1,-1.4,-2.1,carabanchel,5,NaN,NaN,current
4,2020-12-01,2165.0,-0.2,-0.3,-1.3,carabanchel,5,NaN,NaN,current


In [21]:
# Añadimos a la tabla del modelo las variables de Idealista
data_model["monthly_variation"] = float('NaN')
data_model["quarterly_variation"] = float('NaN')
data_model["annual_variation"] = float('NaN')

# Reordenamos tabla
data_model.columns = ["date", "value_type", "price", "conf_low", "conf_up", "Distrito", "geo_element_id", "monthly_variation", "quarterly_variation", "annual_variation"]
data_model = data_model.reindex(columns=["date", "price", "monthly_variation", "quarterly_variation", "annual_variation", "Distrito", "geo_element_id", "conf_low", "conf_up", "value_type"])

data_model.head()

,date,price,monthly_variation,quarterly_variation,annual_variation,Distrito,geo_element_id,conf_low,conf_up,value_type
162,2021-05-01,3964.0,NaN,NaN,NaN,arganzuela,8,3816.215907,4112.531351,prediction
163,2021-06-01,3972.0,NaN,NaN,NaN,arganzuela,8,3823.658580,4119.974024,prediction
164,2021-07-01,3973.0,NaN,NaN,NaN,arganzuela,8,3824.963067,4121.278511,prediction
165,2021-08-01,3957.0,NaN,NaN,NaN,arganzuela,8,3808.711239,4105.026683,prediction
166,2021-09-01,3893.0,NaN,NaN,NaN,arganzuela,8,3744.648015,4040.963459,prediction


In [22]:
# Juntamos tablas
data_joined = data_model.append(data_idealista, ignore_index=True)
data_joined.head()

,date,price,monthly_variation,quarterly_variation,annual_variation,Distrito,geo_element_id,conf_low,conf_up,value_type
0,2021-05-01,3964.0,NaN,NaN,NaN,arganzuela,8,3816.215907,4112.531351,prediction
1,2021-06-01,3972.0,NaN,NaN,NaN,arganzuela,8,3823.658580,4119.974024,prediction
2,2021-07-01,3973.0,NaN,NaN,NaN,arganzuela,8,3824.963067,4121.278511,prediction
3,2021-08-01,3957.0,NaN,NaN,NaN,arganzuela,8,3808.711239,4105.026683,prediction
4,2021-09-01,3893.0,NaN,NaN,NaN,arganzuela,8,3744.648015,4040.963459,prediction


In [23]:
# Eliminamos columna sobrande y guardamos csv
data_joined = data_joined.drop(["Distrito"], axis=1)
data_joined.to_csv("./Historico_Distritos_Predicciones.csv")

## Union Municipio y distritos

## Union Barrios con geo

In [18]:
data_idealista = pd.read_csv("./resumen_barrios.csv")
data_idealista.head()

,Unnamed: 0,Barrio,Precio m2,Variación mensual,Variación trimestral,Variación anual,Máximo histórico,Variación máximo,Distrito
0,0,Tetuán,3664,0.2,0.4,-0.5,3.857 €/m2 dic 2007,-5.0,tetuan
1,1,Bellas Vistas,3489,2.0,0.0,-1.1,3.688 €/m2 mayo 2019,-5.4,tetuan
2,2,Berruguete,3295,-0.3,-1.5,-0.9,3.346 €/m2 feb 2021,-1.5,tetuan
3,3,Cuatro Caminos,4306,-0.2,2.4,2.4,4.480 €/m2 oct 2018,-3.9,tetuan
4,4,Cuzco-Castillejos,4371,-0.6,-0.6,2.0,4.415 €/m2 mayo 2021,-1.0,tetuan


In [19]:
data_geo = pd.read_csv("./geo_elements.csv")
data_geo = data_geo[data_geo.level_id == 3] 
data_geo.head()

,id,name,st_astext,level_id
3,34,Gaztambide,MULTIPOLYGON(((-3.71111051636411 40.4389285909...,3
23,23,Goya,MULTIPOLYGON(((-3.66890004952066 40.4222912235...,3
24,24,Recoletos,MULTIPOLYGON(((-3.69224355357032 40.4195027910...,3
25,25,Fuente del Berro,MULTIPOLYGON(((-3.65904695301188 40.4213291572...,3
26,26,Ventas,MULTIPOLYGON(((-3.62705025142397 40.4153556634...,3


In [20]:
# Comparamos el número de elementos
print(f"{len(data_geo)} {len(data_idealista)}")

131 151


In [23]:
# No encajan los números
len(data_idealista.groupby(["Distrito"]))

21

In [21]:
# Eliminamos los barrios que son distrito
data_idealista["Barrio-norm"] = data_idealista.Barrio.apply(lambda x: unidecode.unidecode(x).lower())

# Vemos si por nombre podemos mapear la mayoría
idealista_districts = {}
for name in data_idealista["Barrio-norm"].unique():
    similar = difflib.get_close_matches(name, data_idealista["Distrito"].unique(), cutoff=0.8)
    if similar:
        idealista_districts[name] = similar[0]
    
idealista_districts

{'tetuan': 'tetuan',
 'ciudad lineal': 'ciudad-lineal',
 'fuencarral': 'fuencarral',
 'villa de vallecas': 'villa-de-vallecas',
 'chamberi': 'chamberi',
 'hortaleza': 'hortaleza',
 'puente de vallecas': 'puente-de-vallecas',
 'villaverde': 'villaverde',
 'arganzuela': 'arganzuela',
 'centro': 'centro',
 'moncloa': 'moncloa',
 'san blas': 'san-blas',
 'latina': 'latina',
 'chamartin': 'chamartin',
 'retiro': 'retiro',
 'salamanca': 'salamanca',
 'usera': 'usera',
 'vicalvaro': 'vicalvaro',
 'barajas': 'barajas',
 'carabanchel': 'carabanchel',
 'moratalaz': 'moratalaz'}

In [22]:
print(f"{len(distritos_idealista)} {len(idealista_districts)}")

22 21


In [23]:
# Ahora vamos a eliminar todos los barrios que son distritos
data_idealista = data_idealista[~data_idealista["Barrio-norm"].isin(list(idealista_districts.keys()))]
len(data_idealista)

130

In [25]:
# Sigue sin encajar el que exista uno menos que en los elementos de geo
data_geo.name.unique()

array(['Gaztambide', 'Goya', 'Recoletos', 'Fuente del Berro', 'Ventas',
       'Argüelles', 'Lista', 'Quintana', 'Pueblo Nuevo', 'Castellana',
       'Almagro', 'Trafalgar', 'Arapiles', 'El Viso', 'Guindalera',
       'Rios Rosas', 'Vallehermoso', 'Acacias', 'San Pascual',
       'Casa de Campo', 'Ciudad Jardín', 'Prosperidad', 'Cuatro Caminos',
       'Orcasitas', 'Palomas', 'Bellas Vistas', 'Hispanoamérica',
       'Piovera', 'Colina', 'Berruguete', 'Alameda de Osuna',
       'Castillejos', 'Valdeacederas', 'Aravaca', 'Atalaya',
       'Nueva España', 'Canillas', 'Almenara', 'Corralejos', 'Valdezarza',
       'Valdemarín', 'Casco Histórico de Barajas', 'Ciudad Universitaria',
       'El Plantío', 'San Cristóbal', 'Pinar del Rey', 'Apostol Santiago',
       'Pacífico', 'Castilla', 'Costillares', 'Pilar', 'Peñagrande',
       'Los Ángeles', 'La Paz', 'Los Rosales', 'Fuentelareina',
       'Palos de Moguer', 'Timón', 'Aeropuerto', 'Valdefuentes',
       'Mirasierra', 'Villaverde Alto, C

In [26]:
# Repitimos distancia de palabras pero con un corte inferior
idealista_geo_names = {}
names_not_found = []
for name in data_idealista["Barrio"].unique():
    similar = difflib.get_close_matches(name, data_geo["name"].unique(), cutoff=0.7)
    if similar:
        idealista_geo_names[name] = similar[0]
    else:
        names_not_found.append(name)
        
len(idealista_geo_names)

111

In [27]:
idealista_geo_names

{'Bellas Vistas': 'Bellas Vistas',
 'Berruguete': 'Berruguete',
 'Cuatro Caminos': 'Cuatro Caminos',
 'Cuzco-Castillejos': 'Castillejos',
 'Valdeacederas': 'Valdeacederas',
 'Colina': 'Colina',
 'Concepción': 'Concepción',
 'Costillares': 'Costillares',
 'Pueblo Nuevo': 'Pueblo Nuevo',
 'Quintana': 'Quintana',
 'San Juan Bautista': 'San Juan Bautista',
 'San Pascual': 'San Pascual',
 'Ventas': 'Ventas',
 'Fuentelarreina': 'Fuentelareina',
 'La Paz': 'La Paz',
 'Mirasierra': 'Mirasierra',
 'Peñagrande': 'Peñagrande',
 'Pilar': 'Pilar',
 'Casco Histórico de Vallecas': 'Casco Histórico de Vallecas',
 'Ensanche de Vallecas - La Gavia': 'Ensanche de Vallecas',
 'Santa Eugenia': 'Santa Eugenia',
 'Almagro': 'Almagro',
 'Arapiles': 'Arapiles',
 'Gaztambide': 'Gaztambide',
 'Trafalgar': 'Trafalgar',
 'Vallehermoso': 'Vallehermoso',
 'Apóstol Santiago': 'Apostol Santiago',
 'Canillas': 'Canillas',
 'Palomas': 'Palomas',
 'Pinar del Rey': 'Pinar del Rey',
 'Entrevías': 'Entrevías',
 'Numancia': 

In [28]:
# Vamos a ver como cuadra los nombres restantes con un punto de corte menor
idealista_geo_names_not_found = {}

for name in names_not_found:
    similar = difflib.get_close_matches(name, data_geo["name"].unique(), cutoff=0.4)
    if similar:
        idealista_geo_names_not_found[name] = similar[0]
    else:
        print(name)
        
idealista_geo_names_not_found

{'Ventilla-Almenara': 'Almenara',
 'Arroyo del Fresno': 'Pinar del Rey',
 'Las Tablas': 'La Paz',
 'Montecarmelo': 'Moscardó',
 'Tres Olivos - Valverde': 'Valverde',
 'Nuevos Ministerios-Ríos Rosas': 'Rios Rosas',
 'Conde Orgaz-Piovera': 'Piovera',
 'Sanchinarro': 'San Isidro',
 'Valdebebas - Valdefuentes': 'Valdefuentes',
 'Virgen del Cortijo - Manoteras': 'Vinateros',
 'San Andrés': 'San Isidro',
 'Chueca-Justicia': 'Justicia',
 'Huertas-Cortes': 'Cortes',
 '12 de Octubre-Orcasur': 'Orcasur',
 'Ambroz': 'Almagro',
 'El Cañaveral - Los Berrocales': 'El Cañaveral',
 'Valdebernardo - Valderribas': 'Valdebernardo',
 'Campo de las Naciones-Corralejos': 'Corralejos',
 'Pau de Carabanchel': 'Puerta del Ángel'}

In [29]:
# Mapeo a mano segun la información anterior
idealista_geo_names['Ventilla-Almenara'] = 'Almenara'
idealista_geo_names['Tres Olivos - Valverde'] = 'Valverde'
idealista_geo_names['Nuevos Ministerios-Ríos Rosas'] = 'Rios Rosas'
idealista_geo_names['Conde Orgaz-Piovera'] = 'Piovera'
idealista_geo_names['Valdebebas - Valdefuentes'] = 'Valdefuentes'
idealista_geo_names['Chueca-Justicia'] = 'Justicia'
idealista_geo_names['Huertas-Cortes'] = 'Cortes'
idealista_geo_names['12 de Octubre-Orcasur'] = 'Orcasur'
idealista_geo_names['El Cañaveral - Los Berrocales'] = 'El Cañaveral'
idealista_geo_names['Valdebernardo - Valderribas'] = 'Valdebernardo'
idealista_geo_names['Campo de las Naciones-Corralejos'] = 'Corralejos'

In [30]:
# Barrops de no mapeados
not_found_names = ['Arroyo del Fresno',
'Las Tablas',
'Montecarmelo',
'Sanchinarro',
'Virgen del Cortijo - Manoteras',
'San Andrés',
'Ambroz',
'Pau de Carabanchel']

In [31]:
idealista_names = list(idealista_geo_names.keys())
geo_names = list(idealista_geo_names.values())

In [32]:
# Comprobamos que no haya duplicados
print(len(idealista_names), len(set(idealista_names)), len(geo_names), len(set(geo_names)))

122 122 122 122


In [33]:
not_found_names

['Arroyo del Fresno',
 'Las Tablas',
 'Montecarmelo',
 'Sanchinarro',
 'Virgen del Cortijo - Manoteras',
 'San Andrés',
 'Ambroz',
 'Pau de Carabanchel']

In [34]:
[item for item in list(data_geo["name"].unique()) if item not in geo_names]

['Atalaya',
 'Aeropuerto',
 'Villaverde Alto, Casco Histórico de Villaverde',
 'El Goloso',
 'El Pardo',
 'Cuatro Vientos',
 'Valderrivas',
 'Atocha',
 'Horcajo']

In [35]:
idealista_geo_names['San Andrés'] = 'Villaverde Alto, Casco Histórico de Villaverde'
idealista_geo_names['Montecarmelo'] = 'El Goloso'

**ASUMIMOS** que la distribución de los barrios de idealista con los oficiales del estado no encajan. Tendremos una serie de barrios sin información. No podemos haer mucho más, demasiado que hemos conseguido ligar tantos.

In [36]:
# Añadimos columna geo para unir fuentes
data_idealista["geo_id"] = data_idealista["Barrio"].apply(lambda x: get_geo_id(idealista_geo_names[x]) if x in idealista_geo_names  else None)
data_idealista.head()

,Unnamed: 0,Barrio,Precio m2,Variación mensual,Variación trimestral,Variación anual,Máximo histórico,Variación máximo,Distrito,Barrio-norm,geo_id
1,1,Bellas Vistas,3489,2.0,0.0,-1.1,3.688 €/m2 mayo 2019,-5.4,tetuan,bellas vistas,47.0
2,2,Berruguete,3295,-0.3,-1.5,-0.9,3.346 €/m2 feb 2021,-1.5,tetuan,berruguete,52.0
3,3,Cuatro Caminos,4306,-0.2,2.4,2.4,4.480 €/m2 oct 2018,-3.9,tetuan,cuatro caminos,44.0
4,4,Cuzco-Castillejos,4371,-0.6,-0.6,2.0,4.415 €/m2 mayo 2021,-1.0,tetuan,cuzco-castillejos,54.0
5,5,Valdeacederas,3262,1.4,-0.2,-0.5,3.541 €/m2 oct 2008,-7.9,tetuan,valdeacederas,55.0


In [37]:
# Comprobamos que tenemos 124 barrios mapeados con geo
data_idealista = data_idealista[~data_idealista["geo_id"].isnull()]
print(len(data_idealista), len(idealista_geo_names))

124 124


In [38]:
# Formateamos fecha
data_idealista["date"] = datetime.strptime('Junio 2021', '%B %Y').date()
data_idealista.head()

,Unnamed: 0,Barrio,Precio m2,Variación mensual,Variación trimestral,Variación anual,Máximo histórico,Variación máximo,Distrito,Barrio-norm,geo_id,date
1,1,Bellas Vistas,3489,2.0,0.0,-1.1,3.688 €/m2 mayo 2019,-5.4,tetuan,bellas vistas,47.0,2021-06-01
2,2,Berruguete,3295,-0.3,-1.5,-0.9,3.346 €/m2 feb 2021,-1.5,tetuan,berruguete,52.0,2021-06-01
3,3,Cuatro Caminos,4306,-0.2,2.4,2.4,4.480 €/m2 oct 2018,-3.9,tetuan,cuatro caminos,44.0,2021-06-01
4,4,Cuzco-Castillejos,4371,-0.6,-0.6,2.0,4.415 €/m2 mayo 2021,-1.0,tetuan,cuzco-castillejos,54.0,2021-06-01
5,5,Valdeacederas,3262,1.4,-0.2,-0.5,3.541 €/m2 oct 2008,-7.9,tetuan,valdeacederas,55.0,2021-06-01


In [39]:
# Mergeamos con GEO
data_merged = data_idealista.merge(data_geo, left_on='geo_id', right_on='id')
# Formateamos
data_merged = data_merged.drop(['Unnamed: 0', 'Máximo histórico', 'id', 'Barrio-norm'], axis=1)
data_merged = data_merged.reindex(columns=["Barrio", "Distrito", "date", "Precio m2", "Variación mensual", "Variación trimestral", "Variación anual", "geo_id", "name", "st_astext", "level_id"])
data_merged.columns = ["idealista_name", "district_name", "date", "price", "monthly_variation", "quarterly_variation", "annual_variation", "geo_element_id", "geo_name", "polygon", "geo_level"]
# Guardamos CSV usado para entrenar modelo
data_merged.to_csv("./Historico_Barrios_geo_detallado.csv")
data_merged.head()

,idealista_name,district_name,date,price,monthly_variation,quarterly_variation,annual_variation,geo_element_id,geo_name,polygon,geo_level
0,Bellas Vistas,tetuan,2021-06-01,3489,2.0,0.0,-1.1,47.0,Bellas Vistas,MULTIPOLYGON(((-3.70811845930643 40.4566756113...,3
1,Berruguete,tetuan,2021-06-01,3295,-0.3,-1.5,-0.9,52.0,Berruguete,MULTIPOLYGON(((-3.70088598297096 40.4582280563...,3
2,Cuatro Caminos,tetuan,2021-06-01,4306,-0.2,2.4,2.4,44.0,Cuatro Caminos,MULTIPOLYGON(((-3.69121051651446 40.4467665489...,3
3,Cuzco-Castillejos,tetuan,2021-06-01,4371,-0.6,-0.6,2.0,54.0,Castillejos,MULTIPOLYGON(((-3.69001937818797 40.4590464424...,3
4,Valdeacederas,tetuan,2021-06-01,3262,1.4,-0.2,-0.5,55.0,Valdeacederas,MULTIPOLYGON(((-3.69445068989253 40.4647281267...,3


In [40]:
data_idealista = data_idealista.drop(['Unnamed: 0', 'Barrio', 'Máximo histórico', 'Variación máximo', 
                                      'Distrito', 'Barrio-norm'], axis=1)
data_idealista.head()

,Precio m2,Variación mensual,Variación trimestral,Variación anual,geo_id,date
1,3489,2.0,0.0,-1.1,47.0,2021-06-01
2,3295,-0.3,-1.5,-0.9,52.0,2021-06-01
3,4306,-0.2,2.4,2.4,44.0,2021-06-01
4,4371,-0.6,-0.6,2.0,54.0,2021-06-01
5,3262,1.4,-0.2,-0.5,55.0,2021-06-01


In [41]:
data_idealista = data_idealista.reindex(columns=["date", "Precio m2", "Variación mensual", "Variación trimestral", "Variación anual", "geo_id"])
data_idealista.head()

,date,Precio m2,Variación mensual,Variación trimestral,Variación anual,geo_id
1,2021-06-01,3489,2.0,0.0,-1.1,47.0
2,2021-06-01,3295,-0.3,-1.5,-0.9,52.0
3,2021-06-01,4306,-0.2,2.4,2.4,44.0
4,2021-06-01,4371,-0.6,-0.6,2.0,54.0
5,2021-06-01,3262,1.4,-0.2,-0.5,55.0


In [42]:
# Cambiamos el nombre
data_idealista.columns = ["date", "price", "monthly_variation", "quarterly_variation", "annual_variation", "geo_element_id"]
data_idealista.head()

,date,price,monthly_variation,quarterly_variation,annual_variation,geo_element_id
1,2021-06-01,3489,2.0,0.0,-1.1,47.0
2,2021-06-01,3295,-0.3,-1.5,-0.9,52.0
3,2021-06-01,4306,-0.2,2.4,2.4,44.0
4,2021-06-01,4371,-0.6,-0.6,2.0,54.0
5,2021-06-01,3262,1.4,-0.2,-0.5,55.0


In [46]:
# Guardamos CSV que se cargara en BBDD
data_idealista.to_csv("./Historico_Barrios_geo.csv")

**IMPORTANTE**


Los barrios que caen fuera son: Atalaya, Aeropuerto, Casco Histórico de Villaverde, El Pardo, Cuatro Vientos, Valderrivas, Atocha, Horcajo